In [20]:
#!pip install -q lxml 

# Librairies

import bs4 
import lxml
import pandas as pd 
import urllib

from urllib import request

# 1. Recupération des données
## A. Les gros succcès au box-office

In [21]:
url_bo = "https://fr.wikipedia.org/wiki/Liste_des_plus_gros_succ%C3%A8s_du_box-office_mondial"
request_text = request.urlopen(url_bo).read()

In [22]:
# Recherche des balises HTML
page = bs4.BeautifulSoup(request_text, "lxml")

In [23]:
# Récupération du tableau Box-Office
tableau_bo = page.find('table', {'class', "wikitable sortable centre"})

# Recupération de chaque ligne du tableau
table_body = tableau_bo.find('tbody')
rows = table_body.find_all('tr')

# Nettoyage de la couche HTML pour ne garder qye le texte utile sur une ligne
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]

# Création d'un dictionnaire avec les infos obtenues 
dico_bo = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : 
        dico_bo[cols[0]] = cols[1:]
#print(dico_bo)

# Création d'un DataFrame avec les infos scrappées sur Wikipédia
data_bo = pd.DataFrame.from_dict(dico_bo, orient = 'index')


In [24]:
# Récupértion des en-têtes de tableau
for row in rows:
    cols = row.find_all('th')
    if len(cols) > 0:
        cols = [ele.get_text(separator= ' ').strip().title() for ele in cols]
        columns_bo = cols
print(columns_bo)

['Rang', 'Meilleur Rang', 'Titre', 'Réalisateur', 'Année', 'Pays De Production', 'Recettes']


In [25]:
data_bo.columns = columns_bo[1:]
data_bo.head()

,Meilleur Rang,Titre,Réalisateur,Année,Pays De Production,Recettes
1,1,Avatar,James Cameron,2009,États-Unis,2 923 706 026 $[a 2]
2,1,Avengers: Endgame,Anthony etJoe Russo,2019,États-Unis,2 799 439 100 $[a 3]
3,3,Avatar : La Voie de l'eau,James Cameron,2022,États-Unis,2 320 250 281 $[a 4]
4,1,Titanic,James Cameron,1997,États-Unis,2 264 743 305 $[a 5]
5,3,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2 071 310 218 $[a 6]


In [26]:
# Nettoyage de la colonne Recettes pour ne garder que le revenu brut
# Et passer d'un str à un int
for i in range(len(data_bo['Recettes'])):
    data_bo['Recettes'][i] = data_bo['Recettes'][i][:14]
# int type dans la colonne Recettes
for i in range(len(data_bo['Recettes'])):
    L = [ele for ele in data_bo['Recettes'][i] if ele!='\xa0']
    data_bo['Recettes'][i] = int(''.join(L))

# Réorganisation des colonnes du DataFrame
data_bo = data_bo[['Titre', 'Réalisateur', 'Année', 'Pays De Production', 'Recettes', 'Meilleur Rang']]
data_bo.head()


,Titre,Réalisateur,Année,Pays De Production,Recettes,Meilleur Rang
1,Avatar,James Cameron,2009,États-Unis,2923706026,1
2,Avengers: Endgame,Anthony etJoe Russo,2019,États-Unis,2799439100,1
3,Avatar : La Voie de l'eau,James Cameron,2022,États-Unis,2320250281,3
4,Titanic,James Cameron,1997,États-Unis,2264743305,1
5,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2071310218,3


## B. Les oscars attribués à ces films

Allez sur la page de chaque film et récupérer les récompenses puis en tirer les oscars. Prendre exemple sur cours du prof

In [27]:
sock = urllib.request.urlopen(url_bo).read()
page_test = bs4.BeautifulSoup(sock)

In [28]:
test = list()
for film in page_test.findAll('i'):
    test.append(film)
test = test[14:64] # On ne garde que les 50 titres de films dans le tableau 

In [29]:
# Récupérer le titre du film (pour après concat les df) et des urls des pages pour les oscars
import re
film = list()
oscar_temp = list()
for ele in test:
    film_url = ele.find('a').get('href')
    film_name = ele.find('a').get('title')
    film.append(film_name)
    url_get_info = 'http://fr.wikipedia.org' + film_url
    #print(url_get_info)

    # Aller sur la page du film
    search = urllib.request.urlopen(url_get_info).read()
    search_film = bs4.BeautifulSoup(search)

    #Trouver les oscars gagnés
    
    awards_section = search_film.find(title=re.compile("Oscar"))
    awards = list()
    if awards_section:
        # Parcourir les éléments suivants pour extraire les récompenses
        for sibling in awards_section.find_next_siblings():
            # Vérifier si nous avons atteint une nouvelle section
            if sibling.name and sibling.name.startswith('h'):
                break
            # Imprimer le texte de l'élément
            awards.append(sibling.get_text().strip())
            print(sibling.get_text().strip())
            
    else:
        awards.append('Empty')
        print("Section des récompenses non trouvée sur la page.")
    #print(awards_section)
    oscar_temp.append(awards)
    print('--- * NEXT MOVIE * ---')


[189]
Meilleure direction artistique pour Rick Carter, Robert Stromberg et Kim Sinclair
Meilleure photographie pour Mauro Fiore
Meilleurs effets visuels pour Joe Letteri, Stephen Rosenbaum, Richard Baneham et Andy Jones
--- * NEXT MOVIE * ---
Meilleurs effets visuels
--- * NEXT MOVIE * ---
meilleur film
décembre 2025
--- * NEXT MOVIE * ---
1998
meilleur film
meilleur réalisateur
[1]
box-office français de tous les temps
[2]
--- * NEXT MOVIE * ---

Harrison Ford

Carrie Fisher
--- * NEXT MOVIE * ---
61e cérémonie des Grammy Awards
Saturn Award
Josh Brolin
--- * NEXT MOVIE * ---
meilleurs effets visuels
--- * NEXT MOVIE * ---
Section des récompenses non trouvée sur la page.
--- * NEXT MOVIE * ---
Meilleurs effets visuels
--- * NEXT MOVIE * ---
--- * NEXT MOVIE * ---
Section des récompenses non trouvée sur la page.
--- * NEXT MOVIE * ---
meilleur son
--- * NEXT MOVIE * ---
Meilleure chanson originale
Into the Unknown
Kristen Anderson-Lopez
Robert Lopez
--- * NEXT MOVIE * ---
Section des r

In [30]:
oscar = list()
for movie in oscar_temp:
    awards = list()
    for ele in movie:
        if "meilleur" in ele.lower():
            awards.append(ele)
        #else: 
            #awards.append('NO')
    oscar.append(awards)

In [31]:
# Création d'un dictionnaire avec les films et leurs oscars respectifs
dico_oscar = dict()
for movie, awards in zip(film, oscar):
        if awards == []:
                dico_oscar[movie] = ['Aucun oscar']
        else: 
                dico_oscar[movie] = awards

In [32]:
dico_oscar

{'Avatar (film, 2009)': ['Meilleure direction artistique pour Rick Carter, Robert Stromberg et Kim Sinclair\nMeilleure photographie pour Mauro Fiore\nMeilleurs effets visuels pour Joe Letteri, Stephen Rosenbaum, Richard Baneham et Andy Jones'],
 'Avengers: Endgame': ['Meilleurs effets visuels'],
 "Avatar : La Voie de l'eau": ['meilleur film'],
 'Titanic (film, 1997)': ['meilleur film', 'meilleur réalisateur'],
 'Star Wars, épisode VII : Le Réveil de la Force': ['Aucun oscar'],
 'Avengers: Infinity War': ['Aucun oscar'],
 'Spider-Man: No Way Home': ['meilleurs effets visuels'],
 'Jurassic World': ['Aucun oscar'],
 'Le Roi lion (film, 2019)': ['Meilleurs effets visuels'],
 'Avengers (film)': ['Aucun oscar'],
 'Fast and Furious 7': ['Aucun oscar'],
 'Top Gun : Maverick': ['meilleur son'],
 'La Reine des neiges 2': ['Meilleure chanson originale'],
 'Barbie (film)': ['Aucun oscar'],
 "Avengers : L'Ère d'Ultron": ['Aucun oscar'],
 'Super Mario Bros. le film': ['Aucun oscar'],
 'Black Panther

In [33]:
data_oscar = pd.DataFrame.from_dict(dico_oscar, orient = 'index')
data_oscar.head()

,0,1,2,3
"Avatar (film, 2009)",Meilleure direction artistique pour Rick Carte...,None,None,None
Avengers: Endgame,Meilleurs effets visuels,None,None,None
Avatar : La Voie de l'eau,meilleur film,None,None,None
"Titanic (film, 1997)",meilleur film,meilleur réalisateur,None,None
"Star Wars, épisode VII : Le Réveil de la Force",Aucun oscar,None,None,None


## C. Utilisation de l'API TMDB pour avoir plus d'infos et récupérer les commentaires

# 2. Analyse des données

## A. Etude descriptive des données en présence

### Idées pour brève description des films en présence
- comparer CA en prenant en compte inflation
- Sur représentation d'un réalisateur, pays origine
- nb d'oscar par film 
- corrélation entre bonnes critiques et nb oscar
- différence entre critique populaire et critique spécialisées ? 

In [34]:
data_bo['Réalisateur'].nunique()

38

In [35]:
data_bo.groupby('Réalisateur')['Titre'].count()

Réalisateur
Aaron HorvathMichael Jelenic (en)    1
Andrew Stanton                       1
Anna BodenRyan Fleck                 1
Anthony etJoe Russo                  3
Bill Condon                          1
Brad Bird                            1
Byron HowardRich Moore               1
Chris BuckJennifer Lee               2
Christopher Nolan                    2
Colin Trevorrow                      1
David Yates                          1
F. Gary Gray                         1
Gareth Edwards                       1
George Lucas                         1
Gore Verbinski                       1
Greta Gerwig                         1
Guy Ritchie                          1
J. J. Abrams                         2
J.A. Bayona                          1
James Cameron                        3
James Wan                            2
Jon Favreau                          1
Jon Watts                            2
Joseph Kosinski                      1
Josh Cooley                          1
Joss Whedon  

In [36]:
data_bo[data_bo['Réalisateur'] == 'J. J. Abrams']

,Titre,Réalisateur,Année,Pays De Production,Recettes,Meilleur Rang
5,"Star Wars, épisode VII : Le Réveil de la Force",J. J. Abrams,2015,États-Unis,2071310218,3
38,"Star Wars, épisode IX : L'Ascension de Skywalker",J. J. Abrams,2019,États-Unis,1077022372,32


# 4. NLP et analyse des commentaires 

# 5. pas d'idée de titre
Mais idée générale: les critiques des amateurs sont-elles en accords avec le fait que les films soient au box-office? 